In [0]:

# # dbutils.widgets.text("file_tm", "")

# # file_name = dbutils.widgets.get("file_tm")
# # input_path = f"abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/landing/{file_name}"

# # df = spark.read.option("multiline", True).json(input_path)

# # output_path = "abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/curated"
# # # Merge or Append logic here
# # df.write.format("delta").mode("append").option("mergeSchema", "true").save(output_path)



# # ---------------------------------------------
# # 1. Get filename passed from ADF pipeline
# # ---------------------------------------------
# dbutils.widgets.text("file_tm", "")
# file_name = dbutils.widgets.get("file_tm")

# # Input file path
# input_path = f"abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/landing/{file_name}"

# # Output folder path
# output_path = "abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/curated"

# # ---------------------------------------------
# # 2. Read the JSON file
# # ---------------------------------------------
# try:
#     df = spark.read.option("multiline", True).json(input_path)

#     if df.isEmpty():
#         print(f"❌ No data found in file: {file_name}")
#     else:
#         # ---------------------------------------------
#         # 3. Append to Delta in curated zone
#         # ---------------------------------------------
#         df.write.format("delta").mode("append").option("mergeSchema", "true").save(output_path)
#         print(f"✅ Data written to curated successfully from file: {file_name}")

#         # ---------------------------------------------
#         # 4. Delete the processed input file
#         # ---------------------------------------------
#         dbutils.fs.rm(input_path, recurse=True)
#         print(f"🗑️ Deleted file from landing: {input_path}")
# except Exception as e:
#     print(f"❌ ERROR processing file: {file_name}")
#     raise e


In [0]:
# from pyspark.sql.functions import col, from_unixtime
# from delta.tables import DeltaTable

# # ---------------------------------------------
# # 1. Get the input file name from ADF
# # ---------------------------------------------
# dbutils.widgets.text("file_tm", "")
# file_name = dbutils.widgets.get("file_tm")

# # Define input and output paths
# input_path = f"abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/landing/{file_name}"
# output_path = "abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/curated/bitcoin_data"

# # ---------------------------------------------
# # 2. Read the JSON file
# # ---------------------------------------------
# try:
#     raw_df = spark.read.option("multiline", True).json(input_path)

#     if raw_df.isEmpty():
#         print(f"No data found in file: {file_name}")
#     else:
#         # ---------------------------------------------
#         # 3. Extract and transform required fields
#         # ---------------------------------------------
#         df = raw_df.selectExpr("explode(data) as record") \
#                    .select(
#                        col("record.priceUsd").cast("double").alias("priceUsd"),
#                        col("record.time").cast("long").alias("time_unix"),
#                        from_unixtime(col("record.time") / 1000).cast("timestamp").alias("timestamp"),
#                        col("record.circulatingSupply").cast("double").alias("circulatingSupply")
#                    )

#         # ---------------------------------------------
#         # 4. Create Delta table if not exists
#         # ---------------------------------------------
#         if not spark._jsparkSession.catalog().tableExists("curated_bitcoin_data"):
#             df.write.format("delta").save(output_path)
#             spark.sql(f"CREATE TABLE curated_bitcoin_data USING DELTA LOCATION '{output_path}'")
#             print(f"Delta table created at: {output_path}")

#         # ---------------------------------------------
#         # 5. Merge into Delta table to prevent duplicates
#         # ---------------------------------------------
#         delta_table = DeltaTable.forPath(spark, output_path)

#         delta_table.alias("target").merge(
#             df.alias("source"),
#             "target.timestamp = source.timestamp"
#         ).whenNotMatchedInsertAll().execute()

#         print(f"Data merged to curated zone successfully from file: {file_name}")

#         # ---------------------------------------------
#         # 6. Delete the processed input file
#         # ---------------------------------------------
#         dbutils.fs.rm(input_path, recurse=True)
#         print(f"Deleted file from landing: {input_path}")

# except Exception as e:
#     print(f"ERROR processing file: {file_name}")
#     raise e

In [0]:
# %python
# from pyspark.sql.functions import col, from_unixtime, to_timestamp
# from delta.tables import DeltaTable

# # ---------------------------------------------
# # 1. Get the input file name from ADF
# # ---------------------------------------------
# dbutils.widgets.text("file_tm", "")
# file_name = dbutils.widgets.get("file_tm")

# # Define input and output paths
# input_path = f"abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/landing/{file_name}"
# output_path = "abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/curated/"

# # ---------------------------------------------
# # 2. Read the JSON file
# # ---------------------------------------------
# try:
#     raw_df = spark.read.option("multiline", True).json(input_path)

#     if raw_df.isEmpty():
#         print(f"No data found in file: {file_name}")
#     else:
#         # ---------------------------------------------
#         # 3. Extract and transform required fields
#         # ---------------------------------------------
#         # df = raw_df.select
#             col("id"),
#             col("symbol"),
#             col("name"),
#             col("current_price").cast("double").alias("price_usd"),
#             col("circulating_supply").cast("double"),
#             col("market_cap").cast("double"),
#             col("total_volume").cast("double"),
#             col("last_updated").alias("timestamp")
#         ).withColumn("timestamp", to_timestamp("timestamp"))

#         # ---------------------------------------------
#         # 4. Create Delta table if not exists
#         # ---------------------------------------------
#         if not spark.catalog.tableExists("curated_bitcoin_data"):
#             df.write.format("delta").save(output_path)
#             spark.sql(f"CREATE TABLE curated_bitcoin_data USING DELTA LOCATION '{output_path}'")
#             print(f"Delta table created at: {output_path}")

#         # ---------------------------------------------
#         # 5. Merge into Delta table to prevent duplicates
#         # ---------------------------------------------
#         delta_table = DeltaTable.forPath(spark, output_path)

#         delta_table.alias("target").merge(
#             df.alias("source"),
#             "target.timestamp = source.timestamp"
#         ).whenNotMatchedInsertAll().execute()

#         print(f"Data merged to curated zone successfully from file: {file_name}")

#         # ---------------------------------------------
#         # 6. Delete the processed input file
#         # ---------------------------------------------
#         dbutils.fs.rm(input_path, recurse=True)
#         print(f"Deleted file from landing: {input_path}")

# except Exception as e:
#     print(f"ERROR processing file: {file_name}")
#     raise e

In [0]:
# %python
# from pyspark.sql.functions import col, to_timestamp, to_date
# from delta.tables import DeltaTable

# # ---------------------------------------------
# # 1. Get the input file name from ADF
# # ---------------------------------------------
# dbutils.widgets.text("file_tm", "")
# file_name = dbutils.widgets.get("file_tm")

# # Define input and output paths
# input_path = f"abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/landing/{file_name}"
# output_path = "abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/curated/"

# # ---------------------------------------------
# # 2. Read the JSON file
# # ---------------------------------------------
# try:
#     raw_df = spark.read.option("multiline", True).json(input_path)

#     if raw_df.isEmpty():
#         print(f"No data found in file: {file_name}")
#     else:
#         # ---------------------------------------------
#         # 3. Extract and transform required fields
#         # ---------------------------------------------
#         df = raw_df.select(
#             col("id"),
#             col("symbol"),
#             col("name"),
#             col("current_price").cast("double").alias("price_usd"),
#             col("circulating_supply").cast("double"),
#             col("market_cap").cast("double"),
#             col("total_volume").cast("double"),
#             col("last_updated").alias("timestamp")
#         ).withColumn("timestamp", to_timestamp("timestamp")) \
#          .withColumn("date", to_date("timestamp"))

#         # ---------------------------------------------
#         # 4. Create Delta table if not exists
#         # ---------------------------------------------
#         if not DeltaTable.isDeltaTable(spark, output_path):
#             df.write.format("delta") \
#                 .mode("overwrite") \
#                 .option("mergeSchema", "true") \
#                 .partitionBy("id", "date") \
#                 .save(output_path)
            
#             spark.sql(f"""
#                 CREATE TABLE IF NOT EXISTS curated_bitcoin_data 
#                 USING DELTA 
#                 LOCATION '{output_path}'
#             """)
#             print(f"Delta table created at: {output_path} (partitioned by id, date)")

#         # ---------------------------------------------
#         # 5. Merge into Delta table to prevent duplicates
#         # ---------------------------------------------
#         delta_table = DeltaTable.forPath(spark, output_path)

#         delta_table.alias("target").merge(
#             df.alias("source"),
#             "target.timestamp = source.timestamp AND target.symbol = source.symbol"
#         ).whenNotMatchedInsertAll().execute()

#         print(f"Data merged to curated zone successfully from file: {file_name}")

#         # ---------------------------------------------
#         # 6. Delete the processed input file
#         # ---------------------------------------------
#         dbutils.fs.rm(input_path, recurse=True)
#         print(f"Deleted file from landing: {input_path}")

# except Exception as e:
#     print(f"ERROR processing file: {file_name}")
#     raise e

In [0]:
%python
from pyspark.sql.functions import col, to_timestamp, to_date, lit, current_timestamp
from delta.tables import DeltaTable

# ---------------------------------------------
# 1. Get the input file name from ADF
# ---------------------------------------------
dbutils.widgets.text("file_tm", "")
file_name = dbutils.widgets.get("file_tm")

# Define input and output paths
input_path = f"abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/landing/{file_name}"
output_path = "abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/curated/"
audit_path = "abfss://cryptoreporting@cryptoreportingadlssa.dfs.core.windows.net/audit/logs"

# ---------------------------------------------
# 2. Read the JSON file
# ---------------------------------------------
try:
    raw_df = spark.read.option("multiline", True).json(input_path)

    if raw_df.isEmpty():
        print(f"No data found in file: {file_name}")
    else:
        # ---------------------------------------------
        # 3. Extract and transform required fields
        # ---------------------------------------------
        df = raw_df.select(
            col("id"),
            col("symbol"),
            col("name"),
            col("current_price").cast("double").alias("price_usd"),
            col("circulating_supply").cast("double"),
            col("market_cap").cast("double"),
            col("total_volume").cast("double"),
            col("last_updated").alias("timestamp")
        ).withColumn("timestamp", to_timestamp("timestamp")) \
         .withColumn("date", to_date("timestamp")) \
         .withColumn("ingestion_time", current_timestamp()) \
         .withColumn("source_file", lit(file_name))

        record_count = df.count()

        # ---------------------------------------------
        # 4. Create Delta table if not exists
        # ---------------------------------------------
        if not DeltaTable.isDeltaTable(spark, output_path):
            df.write.format("delta") \
                .mode("overwrite") \
                .option("mergeSchema", "true") \
                .partitionBy("id", "date") \
                .save(output_path)

            spark.sql(f"""
                CREATE TABLE IF NOT EXISTS curated_bitcoin_data 
                USING DELTA 
                LOCATION '{output_path}'
            """)
            print(f"Delta table created at: {output_path} (partitioned by id, date)")

        # ---------------------------------------------
        # 5. Merge into Delta table to prevent duplicates
        # ---------------------------------------------
        delta_table = DeltaTable.forPath(spark, output_path)

        delta_table.alias("target").merge(
            df.alias("source"),
            "target.timestamp = source.timestamp AND target.symbol = source.symbol"
        ).whenNotMatchedInsertAll().execute()

        print(f"{record_count} records merged successfully from file: {file_name}")

        # ---------------------------------------------
        # 6. Delete the processed input file
        # ---------------------------------------------
        dbutils.fs.rm(input_path, recurse=True)
        print(f"Deleted input file: {input_path}")

        # ---------------------------------------------
        # 7. Log audit details
        # ---------------------------------------------
        audit_df = spark.createDataFrame([{
            "file_name": file_name,
            "record_count": record_count,
            "status": "Success",
            "processed_at": str(current_timestamp())
        }])
        audit_df.write.format("delta").mode("append").save(audit_path)

        # ---------------------------------------------
        # 8. Optimize and Vacuum curated table (optional)
        # ---------------------------------------------
        spark.sql("OPTIMIZE delta.`{}`".format(output_path))
        spark.sql("VACUUM delta.`{}` RETAIN 168 HOURS".format(output_path))  # Retains last 7 days

except Exception as e:
    print(f"ERROR processing file: {file_name}")
    error_df = spark.createDataFrame([{
        "file_name": file_name,
        "record_count": 0,
        "status": f"Failed: {str(e)}",
        "processed_at": str(current_timestamp())
    }])
    error_df.write.format("delta").mode("append").save(audit_path)
    raise e